In [21]:
import pandas as pd
import requests
from elasticsearch import Elasticsearch, helpers
import json
from requests.auth import HTTPBasicAuth
import unidecode
import unicodedata
pd.options.display.max_columns = 100
import numpy as np

### Récupération des données

On récupère les deux sources :
- catalogue dataset
- catalogue orga

In [22]:
# Dataframe catalogue dataset
dfd = pd.read_csv('https://static.data.gouv.fr/resources/catalogue-des-donnees-de-data-gouv-fr/20211007-074534/export-dataset-20211007-074534.csv', dtype=str, sep=";")

In [23]:
# Dataframe catalogue orga
dfo = pd.read_csv('https://static.data.gouv.fr/resources/catalogue-des-donnees-de-data-gouv-fr/20211007-075016/export-organization-20211007-075016.csv', dtype="str", sep=";")

On commence à modifier le dataframe organisation (dfo) avec 2 nouvelles colonnes :
- es_sp : est-ce que l'orga est certifiée service public ou non ?
- es_orga_followers: nombre de followers de l'orga

### Processing des données en amont de l'insertion dans elasticsearch

In [24]:
# On récupère l'info "service public" depuis la colonne badge.
# On attribue la valeur 4 si c'est un SP, 1 si ça ne l'est pas
dfo['es_orga_sp'] = dfo['badges'].apply(lambda x: 4 if 'public-service' in x else 1)

In [25]:
# on sauvegarde en mémoire le dataframe avec uniquement les infos pertinentes
dfo = dfo[['logo','id', 'es_orga_sp', 'metric.followers']]

In [26]:
# On renomme l'id de l'orga et la métrique followers
dfo = dfo.rename(columns={'id':'organization_id', 'metric.followers':'es_orga_followers'})

On merge les deux dataframe (dataset et orga) pour en garder un seul unique

In [27]:
# La colonne de jointure est l'id de l'organisation. On fait un merge de type left join
df = pd.merge(dfd,dfo,on='organization_id',how='left')

On va modifier certaines colonnes pour optimiser par la suite la recherche ES

In [28]:
# On convertit les trois colonnes metric.views metric.followers et es_orga_followers en float
# (elles étaient préalablement des string)
df['metric.views'] = df['metric.views'].astype(float)
df['metric.followers'] = df['metric.followers'].astype(float)
df['es_orga_followers'] = df['es_orga_followers'].astype(float)

Normalisation de certaines colonnes

In [29]:
# Afin qu'une métrique n'est pas plus d'influence que l'autre, on va normaliser chacune des valeurs de ces colonnes
# dans 5 catégorie (de 1 à 5)
# Les "bins" utilisés pour chacune d'entre elles sont préparées "à la main". (difficile en effet d'avoir des bins
# automatiques en sachant que l'immense majorité des datasets ont une valeur de 0 pour ces 3 métriques)
# Comment lire :
# Datasets de 0 vues = valeur 1
# Datasets entre 1 et 49 vues = valeur 2
# Datasets entre 50 et 499 vues = valeur 3
# Datasets entre 500 et 4999 vues = Valeur 4
# Datasets entre 5000 et 'nombre de vues max d'un dataset' = Valeur 5
mvbins = [-1,0,50,500,5000,df['metric.views'].max()]
df['es_dataset_views'] = pd.cut(df['metric.views'], mvbins, labels=list(range(1,6)))
mfbins = [-1,0,2,10,40,df['metric.followers'].max()]
df['es_dataset_followers'] = pd.cut(df['metric.followers'], mfbins, labels=list(range(1,6)))
fobins = [-1,0,10,50,100,df['es_orga_followers'].max()]
df['es_orga_followers'] = pd.cut(df['es_orga_followers'], fobins, labels=list(range(1,6)))

Création d'un champs "es_concat_title_org" concatenant le nom du titre et de l'organisation (car beaucoup de recherche concatène ces deux types de données)

In [30]:
df['es_concat_title_org'] = df['title'] + ' ' + df['organization']

Création d'un champ "es_dataset_featured" se basant sur la colonne features. L'objectif étant de donner un poids plus grand sur les datasets featured

In [31]:
# Poids de 5 quand le dataset est featured, 1 sinon
df['es_dataset_featured'] = df['featured'].apply(lambda x: 5 if x=='True' else 1)

### Insertion des données dans elasticsearch

In [46]:
# On supprime un éventuel index datasets s'il existe
r = requests.delete('http://search.dataeng.etalab.studio/datasets', auth=('elastic','etalab123!'))

In [47]:
# Objet qui sera utilisé plusieurs fois ci-dessous
headers = {
    "Content-Type": "application/json"
}

In [48]:
# On définit un analyzer français (repris ici : https://jolicode.com/blog/construire-un-bon-analyzer-francais-pour-elasticsearch)
# On ajoute dans la filtre french_synonym, des synonymes qu'on souhaite implémenter (ex : AMD / Administrateur des Données)
# On créé notre mapping en indiquant les champs sur lesquels on veut appliquer cet analyzer (title, description, concat, organization)
# et en spécifiant les types des champs qu'on va utiliser pour calculer notre score de pertinence
mapping_with_analyzer = {
  "settings": {
    "analysis": {
      "filter": {
        "french_elision": {
          "type": "elision",
          "articles_case": True,
          "articles": ["l", "m", "t", "qu", "n", "s", "j", "d", "c", "jusqu", "quoiqu", "lorsqu", "puisqu"]
        },
        "french_stop": {
          "type":       "stop",
          "stopwords":  "_french_" 
        },
        "french_synonym": {
          "type": "synonym",
          "ignore_case": True,
          "expand": True,
          "synonyms": [
            "AMD, administrateur ministériel des données, AMDAC",
            "lolf, loi de finance",
            "waldec, RNA, répertoire national des associations",
            "ovq, baromètre des résultats",
            "contour, découpage"
          ]
        },
        "french_stemmer": {
          "type": "stemmer",
          "language": "light_french"
        }
      },
      "analyzer": {
        "french_dgv": {
          "tokenizer": "icu_tokenizer",
          "filter": [
            "french_elision",
            "icu_folding",
            "french_synonym",
            "french_stemmer",
            "french_stop"
          ]
        }
      }
    }
  }, 
  "mappings": {
    "properties": {
      "id": {
        "type": "text"
      },
      "title": {
        "type": "text",
        "analyzer": "french_dgv"
      },
      "description": {
        "type": "text",
        "analyzer": "french_dgv"
      },
      "organization": {
        "type": "text",
        "analyzer": "french_dgv"
      },
      "es_orga_sp": {
        "type": "integer"
      },
      "es_orga_followers": {
        "type": "integer"
      },
      "es_dataset_views": {
        "type": "integer"
      },
      "es_dataset_followers": {
        "type": "integer"
      },
      "es_concat_title_org": {
        "type": "text",
        "analyzer": "french_dgv"
      },
      "es_dataset_featured": {
        "type": "integer"
      },
    
    }
  }
}

On créé l'index "datasets" avec le mapping ci-dessus

In [49]:
r = requests.put('http://search.dataeng.etalab.studio/datasets', auth=('elastic','etalab123!'),
             headers=headers, 
             data=json.dumps(mapping_with_analyzer))

In [52]:
# Vérification que tout s'est bien passé :
assert r.json()['acknowledged'] == True

On index les donnéees en mode bulk (par tranche de 10000 lignes)

In [53]:
%%time
# On lit ligne par ligne le dataframe que l'on envoit à elasticsearch
# l'_id du document est la colonne id
# Peut être robustifié en analysant à chaque requête la réponse pour voir si tout s'est bien passé
df['_id'] = df['id']
df_as_json = df.to_json(orient='records', lines=True)

cpt = 0
final_json_string = ''
for json_document in df_as_json.split('\n'):
    cpt = cpt + 1
    if(json_document != ''):
        jdict = json.loads(json_document)
        metadata = json.dumps({'index': {'_id': jdict['_id']}})
        jdict.pop('_id')
        final_json_string += metadata + '\n' + json.dumps(jdict) + '\n'
    if(cpt % 100 == 0):
        if(cpt % 10000 == 0): 
            print(str(cpt)+' indexed documents')
        headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
        r = requests.post('http://search.dataeng.etalab.studio/datasets/_bulk', data=final_json_string, headers=headers, timeout=60, auth=('elastic','etalab123!'))
        final_json_string = ''
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
r = requests.post('http://search.dataeng.etalab.studio/datasets/_bulk', data=final_json_string, headers=headers, timeout=60, auth=('elastic','etalab123!'))
print(str(cpt)+' indexed documents')



10000 indexed documents
20000 indexed documents
30000 indexed documents
40000 indexed documents
50000 indexed documents
50531 indexed documents


### Recherche de documents

On ordonne les résultats de la recherche de la façon suivante :
Tentative 1 :
- On vérifie que tous les termes ensemble sont présent dans le titre, la description ou l'organisation. Si c'est le cas on met un boost en fonction de :
    - si l'organisation est un service public
    - du nb de vues du dataset
    - du nb de followers du dataset
    - du nb de followers de l'orga
    - si le dataset est featured
Tentative 2 :
- On vérifie que le tous les termes sont présent dans le champs es_dataset_concat. Si c'est le cas on met un boost sur le score (boost plus élevé que la tentative 1). On améliorer le score avec des boost supplémentaire en fonction de :
    - si l'organisation est un service public
    - du nb de vues du dataset
    - du nb de followers du dataset
    - du nb de followers de l'orga
    - si le dataset est featured
Tentative 3 :
- On vérifie que le maximum de termes sont présent dans les champs titre et organization. Cette recherche est de type fuzzy, c'est à dire qu'elle va effectuer une recherche floue à partir des termes pour récupérer plus de résultats. Cette tentative n'a pas de boost.

In [252]:
# Fonction définissant la recherche
def search(terms, detail=False):
    body = {
        "size" : 100,
        "query":{
            "bool":{
                "should":[{
                    "function_score": {
                        "query": {
                            "bool":{  
                              "should":[  
                                {  
                                  "multi_match":{  
                                    "query":terms,
                                    "type":"phrase",
                                    "fields":[
                                        "title^15",
                                        "description^8",
                                        "organization^8"
                                    ]
                                  }
                                }
                              ]
                            }
                          },
                        "functions": [
                            {
                            "field_value_factor": {
                                "field": "es_orga_sp",
                                "factor": 8,
                                "modifier": "sqrt",
                                "missing": 1
                                }
                            },
                            {
                            "field_value_factor": {
                                "field": "es_dataset_views",
                                "factor": 4,
                                "modifier": "sqrt",
                                "missing": 1
                                }
                            },
                            {
                            "field_value_factor": {
                                "field": "es_dataset_followers",
                                "factor": 4,
                                "modifier": "sqrt",
                                "missing": 1
                                }
                            },
                            {
                            "field_value_factor": {
                                "field": "es_orga_followers",
                                "factor": 1,
                                "modifier": "sqrt",
                                "missing": 1
                                }
                            },
                            {
                            "field_value_factor": {
                                "field": "es_dataset_featured",
                                "factor": 1,
                                "modifier": "sqrt",
                                "missing": 1
                                }
                            }
                        ]
                    }
                },
                {
                    "function_score": {
                        "query": {
                            "bool":{  
                              "must":[  
                                {  
                                  "match":{  
                                    "es_concat_title_org": {
                                      "query": terms,
                                      "operator": "and",
                                      "boost": 8
                                     }
                                   }
                                } 
                              ]
                            }
                          },
                        "functions": [
                            {
                            "field_value_factor": {
                                "field": "es_orga_sp",
                                "factor": 8,
                                "modifier": "sqrt",
                                "missing": 1
                                }
                            },
                            {
                            "field_value_factor": {
                                "field": "es_dataset_views",
                                "factor": 4,
                                "modifier": "sqrt",
                                "missing": 1
                                }
                            },
                            {
                            "field_value_factor": {
                                "field": "es_dataset_followers",
                                "factor": 4,
                                "modifier": "sqrt",
                                "missing": 1
                                }
                            },
                            {
                            "field_value_factor": {
                                "field": "es_orga_followers",
                                "factor": 1,
                                "modifier": "sqrt",
                                "missing": 1
                                }
                            },
                            {
                            "field_value_factor": {
                                "field": "es_dataset_featured",
                                "factor": 1,
                                "modifier": "sqrt",
                                "missing": 1
                                }
                            }
                        ]
                    }
                },
                {  
                  "multi_match":{  
                    "query":terms,
                    "type":"most_fields",
                    "fields":[  
                        "title",
                        "organization"
                    ],
                    "fuzziness" : "AUTO"

                  }
                }
                ]
            }
        }
    }
    r = requests.get('http://search.dataeng.etalab.studio/datasets/_search',
             auth=HTTPBasicAuth('elastic','etalab123!'),
             headers=headers, 
             data=json.dumps(body))
    arr = []
    for hit in r.json()['hits']['hits']:
        arr.append(hit)
        if detail: print(str(hit['_score']) + ' - ' + hit['_source']['id'] + ' - ' + hit['_source']['title']+' - '+str(hit['_source']['es_concat_title_org']))

    return arr

Tests de recherche :

In [292]:
arr  = search('dvf', True)

30140.447 - 5d78f093634f411f434cd637 - DVF+ open-data - DVF+ open-data Cerema
21860.71 - 5cc1b94a634f4165e96436c1 - Demandes de valeurs foncières géolocalisées - Demandes de valeurs foncières géolocalisées Etalab
21192.035 - 5c4ae55a634f4117716d5656 - Demandes de valeurs foncières - Demandes de valeurs foncières Ministère de l'économie, des finances et de la relance
18097.861 - 5d11dd269ce2e71faa80cade - Marché immobilier (DVF) - nombre de transactions par année et par commune - Marché immobilier (DVF) - nombre de transactions par année et par commune Communauté d'Agglomération Ventoux Comtat Venaissin
11232.156 - 615bce8666cb384cbbb0681a - Demande de valeurs foncières agrégée à la mutation (DVF+) - Demande de valeurs foncières agrégée à la mutation (DVF+) Grand Poitiers Open Data
8876.422 - 5cdc36a98b4c411a68a6dfef - AppDVF, une application pour suivre le marché immobilier et foncier - AppDVF, une application pour suivre le marché immobilier et foncier Cerema
6492.839 - 5d37d85a9ce2e7

### Benchmark

In [280]:
def bench(url):
    res = requests.get(url)
    data = res.json()
    arr = []
    for q in data['queries']:
        try:
            mydict = {}
            mydict['query'] = q['query']
            mydict['expected'] = q['expected']
            mydict['rank'] = q['rank']
            arr.append(mydict)
        except:
            pass
    dfresult = pd.DataFrame(arr)
    newarr = []
    for index,row in dfresult[dfresult['query'].notna()].iterrows():
        arr = search(row['query'])
        cpt = 0
        found = False
        for a in arr:
            cpt = cpt + 1
            if row['expected'] == a['_source']['id']:
                found = True
                row['new'] = cpt
        if not found:
            row['new'] = None
        newarr.append(row)
    dfresult = pd.DataFrame(newarr)
    # Si non trouvé, on met la valeur 100
    dfresult['rank'] = dfresult['rank'].apply(lambda x: 100 if x != x else x)
    dfresult['new'] = dfresult['new'].apply(lambda x: 100 if x != x else x)
    print('----')
    print('Benchmark - Plus le score est faible, plus la recherche est bonne')
    print('Ancienne recherche, score : '+str(dfresult['rank'].sum()))
    print('Nouvelle recherche, score : '+str(dfresult['new'].sum()))
    return dfresult

In [281]:
pd.options.display.max_rows=300

Benchmark avec le POC en l'état actuel

In [282]:
dfresult = bench('https://raw.githubusercontent.com/maudetes/datagouv-search-indicator/add-matomo-query-dataset/data/poc-recherche.app.etalab.studio/2021-10-08-17-59/queries.json')

----
Benchmark - Plus le score est faible, plus la recherche est bonne
Ancienne recherche, score : 3702.0
Nouvelle recherche, score : 1472.0


Benchmark 1 sur le run de data.gouv.fr

In [289]:
dfresult = bench('https://raw.githubusercontent.com/etalab/datagouv-search-indicator/master/data/www.data.gouv.fr/2021-10-08-17-44/queries.json')

----
Benchmark - Plus le score est faible, plus la recherche est bonne
Ancienne recherche, score : 1526.0
Nouvelle recherche, score : 1489.0


Benchmark 2 sur le run de data.gouv.fr

In [290]:
dfresult = bench('https://raw.githubusercontent.com/maudetes/datagouv-search-indicator/add-matomo-query-dataset/data/www.data.gouv.fr/2021-10-08-17-58/queries.json')

----
Benchmark - Plus le score est faible, plus la recherche est bonne
Ancienne recherche, score : 2689.0
Nouvelle recherche, score : 2542.0


In [291]:
dfresult

,query,expected,rank,new
0,siren,5b7ffc618b4c4169d30727e0,1.0,1.0
1,sirene,5b7ffc618b4c4169d30727e0,1.0,1.0
2,entreprise,5b7ffc618b4c4169d30727e0,8.0,2.0
3,entreprises,5b7ffc618b4c4169d30727e0,8.0,2.0
4,siret,5b7ffc618b4c4169d30727e0,1.0,1.0
5,open damir,54de1e8fc751df388646738b,1.0,1.0
6,opendamir,54de1e8fc751df388646738b,100.0,100.0
7,damir,54de1e8fc751df388646738b,1.0,1.0
8,contours départements,536991b0a3a729239d203d13,3.0,1.0
9,emissions polluantes,53ba4c07a3a729219b7bead3,1.0,10.0
